In [1]:
import polars as pl


# データの読み込み
purchase_df = pl.read_csv("assets/purchase_data.csv")
sale_df = pl.read_csv("assets/sale_data.csv")

In [2]:
# `purchase_data`の表示
display(purchase_df)

売上日,時刻,ユーザー,購入金額
str,i64,str,i64
"""2025-01-01""",9,"""A""",3510
"""2025-01-01""",9,"""I""",2100
"""2025-01-01""",13,"""C""",4210
"""2025-01-01""",13,"""M""",1850
"""2025-01-01""",14,"""A""",2770
…,…,…,…
"""2025-03-31""",19,"""H""",3250
"""2025-03-31""",19,"""W""",3960
"""2025-03-31""",20,"""A""",2960


In [3]:
# `sale_data`の表示
display(sale_df)

日時
str
"""2025-01-28T16:00:00.000000"""
"""2025-01-24T18:00:00.000000"""
"""2025-03-13T09:00:00.000000"""
"""2025-01-09T21:00:00.000000"""
"""2025-02-16T10:00:00.000000"""
"""2025-02-02T10:00:00.000000"""
"""2025-03-03T11:00:00.000000"""
"""2025-01-19T21:00:00.000000"""
"""2025-03-08T14:00:00.000000"""


## 1. 文字列から`Datetime型`への変換

`purchase_df`の**売上日**および**時刻**からPolarsのDatetime型へ変換して、新しく**日時**列を追加してください。

Polarsでは、文字列型のstrアクセラの`to_datetime`を用いることで日時データを文字列型からDatetime型へ変換することができます。  
変換時に必要なフォーマット(今回だと`"%Y-%m-%d"`)は、[chrono crate documentation](https://docs.rs/chrono/latest/chrono/format/strftime/index.html)を参考にしてください。

`Datetime型`に時間(hour)や日にち(day)を加える場合は、`pl.duration`で`Duration型`に変換することで加減算することができます。

In [4]:
purchase_df = pl.read_csv("assets/purchase_data.csv")
purchase_df = purchase_df.with_columns(
    (pl.col("売上日").str.to_datetime("%Y-%m-%d") + pl.duration(hours=pl.col("時刻"))).alias("日時")
)
display(purchase_df)

売上日,時刻,ユーザー,購入金額,日時
str,i64,str,i64,datetime[μs]
"""2025-01-01""",9,"""A""",3510,2025-01-01 09:00:00
"""2025-01-01""",9,"""I""",2100,2025-01-01 09:00:00
"""2025-01-01""",13,"""C""",4210,2025-01-01 13:00:00
"""2025-01-01""",13,"""M""",1850,2025-01-01 13:00:00
"""2025-01-01""",14,"""A""",2770,2025-01-01 14:00:00
…,…,…,…,…
"""2025-03-31""",19,"""H""",3250,2025-03-31 19:00:00
"""2025-03-31""",19,"""W""",3960,2025-03-31 19:00:00
"""2025-03-31""",20,"""A""",2960,2025-03-31 20:00:00


また、Polarsだとデータの読み込み時に` try_parse_dates=True`を指定することで、`date型`や`Datetime型`へ推定させることができます。  

In [5]:
purchase_df = pl.read_csv("assets/purchase_data.csv", try_parse_dates=True)
purchase_df = purchase_df.with_columns(
    (pl.col("売上日").cast(pl.Datetime("us")) + pl.duration(hours=pl.col("時刻"))).alias("日時")
)
display(purchase_df)

売上日,時刻,ユーザー,購入金額,日時
date,i64,str,i64,datetime[μs]
2025-01-01,9,"""A""",3510,2025-01-01 09:00:00
2025-01-01,9,"""I""",2100,2025-01-01 09:00:00
2025-01-01,13,"""C""",4210,2025-01-01 13:00:00
2025-01-01,13,"""M""",1850,2025-01-01 13:00:00
2025-01-01,14,"""A""",2770,2025-01-01 14:00:00
…,…,…,…,…
2025-03-31,19,"""H""",3250,2025-03-31 19:00:00
2025-03-31,19,"""W""",3960,2025-03-31 19:00:00
2025-03-31,20,"""A""",2960,2025-03-31 20:00:00


In [6]:
sale_df = pl.read_csv("assets/sale_data.csv", try_parse_dates=True)
display(sale_df)

日時
datetime[μs]
2025-01-28 16:00:00
2025-01-24 18:00:00
2025-03-13 09:00:00
2025-01-09 21:00:00
2025-02-16 10:00:00
2025-02-02 10:00:00
2025-03-03 11:00:00
2025-01-19 21:00:00
2025-03-08 14:00:00


## 2. 土日の午後1時から午後5時までの行動を抽出する

`purchase_df`から土日の午後1時から午後5時までのデータを選択して、新しいデータフレーム`new_purchase_df`を作成してください。

`Datetime型`はdtアクセサの`.hour()`で時間を、`.weekday()`で1を月曜日、7を日曜日にした曜日を取得することができます。

tipsですが、上限と下限が決められている場合は`is_between`で、要素が含まれているかを`is_in`で判定することができてシンプルに記述することができます。

In [7]:
purchase_df = pl.read_csv("assets/purchase_data.csv")
purchase_df = purchase_df.with_columns(
    (pl.col("売上日").str.to_datetime("%Y-%m-%d") + pl.duration(hours=pl.col("時刻"))).alias("日時")
)
new_purchase_df = purchase_df.filter(
    (pl.col("日時").dt.hour().is_between(13, 17, closed="both"))
    & (pl.col("日時").dt.weekday().is_in([6, 7]))  # 6が土曜日, 7が日曜日
)
display(new_purchase_df)

売上日,時刻,ユーザー,購入金額,日時
str,i64,str,i64,datetime[μs]
"""2025-01-04""",14,"""G""",3880,2025-01-04 14:00:00
"""2025-01-04""",14,"""V""",1730,2025-01-04 14:00:00
"""2025-01-04""",17,"""R""",2680,2025-01-04 17:00:00
"""2025-01-04""",17,"""U""",3200,2025-01-04 17:00:00
"""2025-01-05""",13,"""W""",3110,2025-01-05 13:00:00
…,…,…,…,…
"""2025-03-29""",17,"""T""",3720,2025-03-29 17:00:00
"""2025-03-30""",13,"""C""",3320,2025-03-30 13:00:00
"""2025-03-30""",16,"""I""",1820,2025-03-30 16:00:00


## 3. 何日ぶりに購入したかを計算する

`purchase_df`でユーザーが何日ぶりに訪れたかを、新しく**来店間隔**列を作成してください。

Polarsでは`shift`と`over`を使用することでグループごとにシフトすることができます。  
今回では `pl.col("日時").shift(1).over("ユーザー")` とすることで1つ前の日時を取得することができます(なおソートされていることが前提です)。  
なお、Nステップ前との差分という処理であれば`shift`として差分を取るのと、`diff`を実行するのは同様の操作になります。

次に`Datetime型`同士で差分を計算することが可能で、`Duration型`が返されます。  
`Duration型`もdtアクササがありますが、`Datetime型`と異なることは注意が必要です。  
今回のケースだと、日にちを計算するために`total_days`を使用します。

In [8]:
purchase_df = pl.read_csv("assets/purchase_data.csv")
purchase_df = purchase_df.with_columns(
    (pl.col("売上日").str.to_datetime("%Y-%m-%d") + pl.duration(hours=pl.col("時刻"))).alias("日時")
)

purchase_df = purchase_df.sort(["日時"])  # もしデータがソートされていない場合はソートする
purchase_df = purchase_df.with_columns(
    (pl.col("日時").diff(1).over("ユーザー")).dt.total_days().alias("来店間隔")
)
display(purchase_df)

売上日,時刻,ユーザー,購入金額,日時,来店間隔
str,i64,str,i64,datetime[μs],i64
"""2025-01-01""",9,"""A""",3510,2025-01-01 09:00:00,null
"""2025-01-01""",9,"""I""",2100,2025-01-01 09:00:00,null
"""2025-01-01""",13,"""C""",4210,2025-01-01 13:00:00,null
"""2025-01-01""",13,"""M""",1850,2025-01-01 13:00:00,null
"""2025-01-01""",14,"""A""",2770,2025-01-01 14:00:00,0
…,…,…,…,…,…
"""2025-03-31""",19,"""H""",3250,2025-03-31 19:00:00,3
"""2025-03-31""",19,"""W""",3960,2025-03-31 19:00:00,0
"""2025-03-31""",20,"""A""",2960,2025-03-31 20:00:00,0


## 4. 現在の行を含めずにユーザーごとに累計購入金額と累計購入回数を計算する

`purchase_df`で各ユーザーの累計購入金額および累計購入回数を計算して**累計購入金額**, **累計来店回数**列を追加してください。  
ただし同一の行の情報は含まないようにして、**累計来店回数**においてもしこれまで来店したことがなければ0の値にしてください。

Polarsでは、累積和は`cum_sum`、累積カウントは`cum_count`で、`over句`を使うことでグループごとに集約することができます。  
また、同一行の情報は含まないようにするために、1つshiftする必要があります。  
`pl.col("購入金額").cum_sum().shift(1).over("ユーザー")`とすることで、累積和を計算して1つshiftする処理をシンプルに記述することができます。

In [9]:
purchase_df = pl.read_csv("assets/purchase_data.csv")
purchase_df = purchase_df.with_columns(
    (pl.col("購入金額").cum_sum().shift(1).over("ユーザー")).alias("累計購入金額"),
    (pl.col("購入金額").cum_count().shift(1).over("ユーザー").fill_null(0)).alias("累計来店回数"),
)
display(purchase_df)

売上日,時刻,ユーザー,購入金額,累計購入金額,累計来店回数
str,i64,str,i64,i64,u32
"""2025-01-01""",9,"""A""",3510,null,0
"""2025-01-01""",9,"""I""",2100,null,0
"""2025-01-01""",13,"""C""",4210,null,0
"""2025-01-01""",13,"""M""",1850,null,0
"""2025-01-01""",14,"""A""",2770,3510,1
…,…,…,…,…,…
"""2025-03-31""",19,"""H""",3250,129510,42
"""2025-03-31""",19,"""W""",3960,158530,42
"""2025-03-31""",20,"""A""",2960,114890,34


## 5. 直近3回の購入平均額および最大購入額を計算する

`purchase_df`で各ユーザーごとの直近3回の購入金額の平均と、最大購入金額を計算して**直近3回の購入平均額**, **直近3回の最大購入額**列を追加してください。  
ただしもし3回も購入したことがなければ、最低でも**2回以上**購入していればその平均値と最大値の計算を行ってください。

Polarsでは、**rolling_mean**や**rolling_max**などでNステップ前までの集約を行うことができます。  
また、`min_samples`を指定することが可能で、最低何個のサンプルが存在すれば計算を行うか指定することができます。  
記述方法としては先ほどの問題とほぼ一緒で、`cum_sum`から`rolling_mean`, `rolling_max`に変更するだけです。

`mean`や`max`の他にも`min`, `max`, `std`などの集約も可能なため、公式ドキュメントを確認してください。

In [10]:
purchase_df = pl.read_csv("assets/purchase_data.csv")

purchase_df = purchase_df.with_columns(
    (pl.col("購入金額").rolling_mean(3, min_samples=2).shift(1).over("ユーザー")).alias(
        "直近3回の購入平均額"
    ),
    (pl.col("購入金額").rolling_max(3, min_samples=2).shift(1).over("ユーザー")).alias(
        "直近3回の最大購入額"
    ),
)
display(purchase_df)

売上日,時刻,ユーザー,購入金額,直近3回の購入平均額,直近3回の最大購入額
str,i64,str,i64,f64,i64
"""2025-01-01""",9,"""A""",3510,null,null
"""2025-01-01""",9,"""I""",2100,null,null
"""2025-01-01""",13,"""C""",4210,null,null
"""2025-01-01""",13,"""M""",1850,null,null
"""2025-01-01""",14,"""A""",2770,null,null
…,…,…,…,…,…
"""2025-03-31""",19,"""H""",3250,2826.666667,3610
"""2025-03-31""",19,"""W""",3960,4126.666667,4270
"""2025-03-31""",20,"""A""",2960,3523.333333,3810


ユーザーAに限定すると以下のように計算される。

In [11]:
display(purchase_df.filter(pl.col("ユーザー") == "A").head(5))

売上日,時刻,ユーザー,購入金額,直近3回の購入平均額,直近3回の最大購入額
str,i64,str,i64,f64,i64
"""2025-01-01""",9,"""A""",3510,null,null
"""2025-01-01""",14,"""A""",2770,null,null
"""2025-01-01""",21,"""A""",3440,3140.0,3510
"""2025-01-02""",9,"""A""",2910,3240.0,3510
"""2025-01-04""",20,"""A""",2340,3040.0,3440


## 6. 過去1ヶ月の平均購入額を計算する

`purchase_df`で各ユーザーで直近1ヶ月の平均購入金額を計算して**直近1ヶ月の購入平均額**列を追加してください。  
ただし同一の行の情報は含まないようにしてください。

Polarsでは、`rolling_mean_by`を用いて、時系列情報の列を`by`で指定して、`window_size`で集約したい時間幅を指定することで処理することができます。  
また、`closed`で一致する両端を集約に含めるかを選択することができます。  
今回の場合だと同一の行情報は含まないようにしたいので右側を含めない`left`に指定します。

In [12]:
purchase_df = pl.read_csv("assets/purchase_data.csv")
purchase_df = purchase_df.with_columns(
    (pl.col("売上日").str.to_datetime("%Y-%m-%d") + pl.duration(hours=pl.col("時刻"))).alias("日時")
)

purchase_df = purchase_df.with_columns(
    (
        pl.col("購入金額")
        .rolling_mean_by(by="日時", window_size="1mo", closed="left")
        .over("ユーザー")
    ).alias("直近1ヶ月の購入平均額")
)
display(purchase_df)

売上日,時刻,ユーザー,購入金額,日時,直近1ヶ月の購入平均額
str,i64,str,i64,datetime[μs],f64
"""2025-01-01""",9,"""A""",3510,2025-01-01 09:00:00,null
"""2025-01-01""",9,"""I""",2100,2025-01-01 09:00:00,null
"""2025-01-01""",13,"""C""",4210,2025-01-01 13:00:00,null
"""2025-01-01""",13,"""M""",1850,2025-01-01 13:00:00,null
"""2025-01-01""",14,"""A""",2770,2025-01-01 14:00:00,3510.0
…,…,…,…,…,…
"""2025-03-31""",19,"""H""",3250,2025-03-31 19:00:00,2858.0
"""2025-03-31""",19,"""W""",3960,2025-03-31 19:00:00,3876.666667
"""2025-03-31""",20,"""A""",2960,2025-03-31 20:00:00,3559.333333


なお、これまでPolarsのExpressionで記述できるメソッドを紹介していましたが、rollingは`pl.DataFrame`に対するメソッドが存在します。

`rolling_mean_by`に対して`index_column`が`by`に相当して、`group_by`でグループを指定し、`period`で時間幅を指定します。  
さらに`.agg`で集約式を記述することで目的の移動平均を計算することが可能です。

では、新しく計算した`rolling_df`を`purchase_df`に結合するとどうなるでしょうか…？

In [13]:
purchase_df = pl.read_csv("assets/purchase_data.csv")
purchase_df = purchase_df.with_columns(
    (pl.col("売上日").str.to_datetime("%Y-%m-%d") + pl.duration(hours=pl.col("時刻"))).alias("日時")
)

rolling_df = (
    purchase_df.rolling(
        index_column="日時",
        group_by="ユーザー",
        period="1mo",
        closed="left",
    )
    .agg(pl.col("購入金額").mean())
    .rename({"購入金額": "直近1ヶ月の購入平均額"})
)

# purchase_dfに結合?
purchase_df = purchase_df.join(rolling_df, on=["日時", "ユーザー"], how="left")
display(purchase_df)

売上日,時刻,ユーザー,購入金額,日時,直近1ヶ月の購入平均額
str,i64,str,i64,datetime[μs],f64
"""2025-01-01""",9,"""A""",3510,2025-01-01 09:00:00,null
"""2025-01-01""",9,"""I""",2100,2025-01-01 09:00:00,null
"""2025-01-01""",13,"""C""",4210,2025-01-01 13:00:00,null
"""2025-01-01""",13,"""M""",1850,2025-01-01 13:00:00,null
"""2025-01-01""",14,"""A""",2770,2025-01-01 14:00:00,3510.0
…,…,…,…,…,…
"""2025-03-31""",19,"""H""",3250,2025-03-31 19:00:00,2858.0
"""2025-03-31""",19,"""W""",3960,2025-03-31 19:00:00,3876.666667
"""2025-03-31""",20,"""A""",2960,2025-03-31 20:00:00,3559.333333


なんと行数が1,000から増えてしまっています。  
原因としては、重複する`売上日`および`ユーザー`があることです。`.unique`で重複する行を削除して結合することで解決します。

In [14]:
purchase_df = pl.read_csv("assets/purchase_data.csv")
purchase_df = purchase_df.with_columns(
    (pl.col("売上日").str.to_datetime("%Y-%m-%d") + pl.duration(hours=pl.col("時刻"))).alias("日時")
)

rolling_df = (
    purchase_df.rolling(
        index_column="日時",
        group_by="ユーザー",
        period="1mo",
        closed="left",
    )
    .agg(pl.col("購入金額").mean())
    .rename({"購入金額": "直近1ヶ月の購入平均額"})
)

# 重複を取り除く
rolling_df = rolling_df.unique(subset=["日時", "ユーザー"])

# purchase_dfに結合
purchase_df = purchase_df.join(rolling_df, on=["日時", "ユーザー"], how="left")
display(purchase_df)

売上日,時刻,ユーザー,購入金額,日時,直近1ヶ月の購入平均額
str,i64,str,i64,datetime[μs],f64
"""2025-01-01""",9,"""A""",3510,2025-01-01 09:00:00,null
"""2025-01-01""",9,"""I""",2100,2025-01-01 09:00:00,null
"""2025-01-01""",13,"""C""",4210,2025-01-01 13:00:00,null
"""2025-01-01""",13,"""M""",1850,2025-01-01 13:00:00,null
"""2025-01-01""",14,"""A""",2770,2025-01-01 14:00:00,3510.0
…,…,…,…,…,…
"""2025-03-31""",19,"""H""",3250,2025-03-31 19:00:00,2858.0
"""2025-03-31""",19,"""W""",3960,2025-03-31 19:00:00,3876.666667
"""2025-03-31""",20,"""A""",2960,2025-03-31 20:00:00,3559.333333


## 7. 過去1週間から過去1日前までの平均購入額を計算する

`purchase_df`で各ユーザーで直近1ヶ月の平均購入金額を計算して**過去1週間から過去1日前までの購入平均額**列を追加してください。

`rolling`メソッドには、`period`の他に`offset`を指定することができます。  
`offset`を指定したときの範囲は`index_column`で指定した日時を`t`としたとき

$$
(t + \text{offset},\ t + \text{offset} + \text{period}]
$$

そのため、過去1週間前から過去1日までを指定する場合は、`offset`を`-1w`, `period`を`6d`に指定する必要があります。

In [15]:
purchase_df = pl.read_csv("assets/purchase_data.csv")
purchase_df = purchase_df.with_columns(
    (pl.col("売上日").str.to_datetime("%Y-%m-%d") + pl.duration(hours=pl.col("時刻"))).alias("日時")
)

rolling_df = (
    purchase_df.rolling(
        index_column="日時",
        group_by="ユーザー",
        period="6d",
        offset="-1w",
        closed="both",
    )
    .agg(pl.col("購入金額").mean())
    .rename({"購入金額": "過去1週間から過去1日前までの購入平均額"})
)

rolling_df = rolling_df.unique(subset=["日時", "ユーザー"])

purchase_df = purchase_df.join(rolling_df, on=["日時", "ユーザー"], how="left")
display(purchase_df)

売上日,時刻,ユーザー,購入金額,日時,過去1週間から過去1日前までの購入平均額
str,i64,str,i64,datetime[μs],f64
"""2025-01-01""",9,"""A""",3510,2025-01-01 09:00:00,null
"""2025-01-01""",9,"""I""",2100,2025-01-01 09:00:00,null
"""2025-01-01""",13,"""C""",4210,2025-01-01 13:00:00,null
"""2025-01-01""",13,"""M""",1850,2025-01-01 13:00:00,null
"""2025-01-01""",14,"""A""",2770,2025-01-01 14:00:00,null
…,…,…,…,…,…
"""2025-03-31""",19,"""H""",3250,2025-03-31 19:00:00,2830.0
"""2025-03-31""",19,"""W""",3960,2025-03-31 19:00:00,3980.0
"""2025-03-31""",20,"""A""",2960,2025-03-31 20:00:00,3550.0


## 8. 各日にちの合計購入額を計算する

`purchase_df`で2025-01-01から2025-03-31まで各日にちごとに購入金額を合計した**日時**と**日次購入金額**からなる新しいDataFrameを作成してください。

Polarsでは、`rolling`などでは行に対して集約したのに対し、`group_by_dynamic`で特定の日時基準で集約することができます。  
`index_column`で日時を指定して、`every`で集約間隔、`period`で集約範囲を指定することができます。

In [16]:
purchase_df = pl.read_csv("assets/purchase_data.csv")
purchase_df = purchase_df.with_columns(
    (pl.col("売上日").str.to_datetime("%Y-%m-%d") + pl.duration(hours=pl.col("時刻"))).alias("日時")
)

daily_df = (
    purchase_df.group_by_dynamic(index_column="日時", every="1d", period="1d")
    .agg(pl.col("購入金額").sum())
    .rename({"購入金額": "日次購入金額"})
)
display(daily_df)

日時,日次購入金額
datetime[μs],i64
2025-01-01 00:00:00,50890
2025-01-02 00:00:00,48840
2025-01-03 00:00:00,41470
2025-01-04 00:00:00,36590
2025-01-05 00:00:00,39600
…,…
2025-03-27 00:00:00,45160
2025-03-28 00:00:00,26550
2025-03-29 00:00:00,28960
